In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv',delimiter = '|')

In [3]:
df.columns

Index(['image_name', ' comment_number', ' comment'], dtype='object')

In [4]:
df.rename(columns={'image_name': 'image_name', ' comment_number': 'comment_number', ' comment': 'comment'}, inplace=True)

In [5]:
df.dtypes

image_name        object
comment_number    object
comment           object
dtype: object

In [6]:
# Function to create a mapping of images to captions
def create_image_caption_mapping(df):
    image_caption_map = {}
    for _, row in df.iterrows():
        img_name, caption = row['image_name'], row['comment']
        if pd.isna(caption):
            continue  # Skip NaN captions
        caption = caption.strip()
        
        if img_name not in image_caption_map:
            image_caption_map[img_name] = []
        image_caption_map[img_name].append(caption)
    return image_caption_map


In [8]:
from sklearn.model_selection import train_test_split


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [41]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_data = create_image_caption_mapping(train_df)
test_data = create_image_caption_mapping(test_df)


In [10]:
!pip install transformers timm

In [43]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained('Salesforce/blip-image-captioning-base')
model = BlipForConditionalGeneration.from_pretrained('Salesforce/blip-image-captioning-base')


In [44]:
from PIL import Image

def load_image(image_path):
    return Image.open(image_path)


In [45]:
def generate_caption(image_path, processor, model):
    image = load_image(image_path)
    inputs = processor(image, return_tensors="pt")
    outputs = model.generate(**inputs)
    return processor.decode(outputs[0], skip_special_tokens=True)


In [42]:
test_data = dict(list(test_data.items())[:1000])

In [62]:
from nltk.translate.bleu_score import corpus_bleu

actual, predicted = [], []
c = 0
for img_path, caption_list in test_data.items():
    c+=1
    image_full_path = '/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/' + img_path
    generated_caption = generate_caption(image_full_path, processor, model)
    actual_captions = [caption.split() for caption in caption_list]
    predicted_caption = generated_caption.split()
    actual.append(actual_captions)
    predicted.append([predicted_caption])

In [57]:
print(len(predicted))

843


In [58]:
print(len(actual))

843


In [59]:
predicted = [item[0] for item in predicted]

In [60]:
print(predicted[0])

['a', 'man', 'playing', 'a', 'guitar']


In [61]:
bleu_score = corpus_bleu(actual, predicted)

BLEU Score: 0.05057499244444994
